# Árvores II - Tarefa 2

### 1. Carregar as bases

Vamos carregar as bases lidas na tarefa passada. Se você salvou essas bases em arquivo texto, basta fazer a leitura com o comando ```pd.read_csv``` das seguintes bases:

- X_train
- Y_train
- X_test
- Y_test

Não se esqueça de considerar a leitura dos índices dos arquivos no ```read_csv()```!

In [5]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix

In [6]:
# Carregando a base features.txt em uma Series
with open('features.txt', 'r') as file:
    lines = file.readlines()
    features_series = pd.Series(lines).str.rstrip('\n')

# Carregando a base subject_train.txt em uma Series
subject_train_series = pd.read_csv("subject_train.txt", header=None)

# Carregando a base X_train.txt em um DataFrame
X_train = pd.read_csv('X_train.txt', delim_whitespace=True, header=None)

# Definindo os nomes das colunas com base em features.txt
X_train.columns = features_series.tolist()

# Adicionando a coluna do identificador do indivíduo a X_train_df
X_train['subject_train_series'] = subject_train_series

# Criando um índice duplo com a ordem dos dados e o identificador do indivíduo
X_train.set_index([X_train.index, 'subject_train_series'], inplace=True)

# Carregando 'subject_test.txt' em uma Series
subject_test_series = pd.read_csv('subject_test.txt', header=None)

# Carregando 'X_test.txt' em um DataFrame
X_test = pd.read_csv('X_test.txt', delim_whitespace=True, header=None)

# Definindo os nomes das colunas com base em features.txt
X_test.columns = features_series.tolist()

# Adicionando a coluna subject_test como uma coluna em X_test_df
X_test['subject_test'] = subject_test_series

# Criando um índice duplo com a ordem dos dados e o identificador do indivíduo
X_test.set_index([X_test.index, 'subject_test'], inplace=True)

# Carregando os dados de treinamento
y_train = pd.read_csv('y_train.txt')

print(X_train)
print(X_test)

                           1 tBodyAcc-mean()-X  2 tBodyAcc-mean()-Y  \
     subject_train_series                                             
0    1                                0.288585            -0.020294   
1    1                                0.278419            -0.016411   
2    1                                0.279653            -0.019467   
3    1                                0.279174            -0.026201   
4    1                                0.276629            -0.016570   
...                                        ...                  ...   
7347 30                               0.299665            -0.057193   
7348 30                               0.273853            -0.007749   
7349 30                               0.273387            -0.017011   
7350 30                               0.289654            -0.018843   
7351 30                               0.351503            -0.012423   

                           3 tBodyAcc-mean()-Z  4 tBodyAcc-std()-X  \
     s

### 2. Divisão da base em Treino, Validação e Teste

A base já se encontra dividida em Treino e Validação. O que vamos fazer então é extrair uma base de Validação da base de Treino.

Extraia 25% da base de treino como base de validação.

In [19]:
# Define a porcentagem de 25% para a base de validação.
proporcao_validacao = 0.25

# Divide a base de treinamento em base de treinamento e base de validação
X_treino, X_validacao, y_treino, y_validacao = train_test_split(X_train, y_train, test_size=proporcao_validacao, random_state=42)

print(X_treino)

                           1 tBodyAcc-mean()-X  2 tBodyAcc-mean()-Y  \
     subject_train_series                                             
6593 28                               0.217295            -0.028434   
3519 17                               0.294986            -0.009296   
4376 22                               0.277687            -0.021567   
4294 21                               0.284126            -0.016949   
29   1                                0.279998            -0.019484   
...                                        ...                  ...   
5191 25                               0.278897            -0.030306   
5226 25                               0.289183            -0.049248   
5390 25                               0.293946            -0.018341   
860  5                                0.280475            -0.018976   
7270 30                               0.263582             0.006928   

                           3 tBodyAcc-mean()-Z  4 tBodyAcc-std()-X  \
     s

### 3. Melhores 3 variáveis

Rode uma árvore com profundidade máxima igual a 4 para prever a atividade humana com todas as variáveis.
Observe a importância das variáveis e considere as 3 variáveis com maior importância para os próximos passos.
Dica: utilize o atributo ```clf.feature_importances_``` da árvore treinada.

In [21]:
# Crie um modelo de árvore de decisão com profundidade máxima igual a 4
dtc_1 = DecisionTreeClassifier(max_depth=4, random_state=42)

# Treine o modelo com os dados de treinamento
dtc_1.fit(X_treino, y_treino)

# Obtendo a importancia das variáveis
importancias_variaveis = dtc_1.feature_importances_

# Crie um dicionário que associe as importâncias às colunas
importancias = dict(zip(X_treino.columns, importancias_variaveis))

# Ordenando as importâncias em ordem decrescente
variaveis_importantes = sorted(importancias.items(), key=lambda x: x[1], reverse=True)

# Selecione as três variáveis mais importantes, porém só com os nomes para evitar erros durante o treino
top_3_variaveis = [variavel for variavel, _ in variaveis_importantes[:3]]

top_3_variaveis

['53 tGravityAcc-min()-X',
 '390 fBodyAccJerk-bandsEnergy()-1,16',
 '560 angle(Y,gravityMean)']

### 4. Construa uma árvore com as 3 melhores variáveis

Utilizando as três variáveis encontradas acima, construa uma árvore de decisão. Encontre o melhor ```ccp_alpha``` utilizando a base de validação, conforme a estrutura que vimos em aula.

In [35]:
# Criando uma lista para armazenar os ccp_alphas
ccp_alphas = np.linspace(0, 0.1, 100)

# Lista para armazenar as acurácias da base de validação para cada ccp_alpha
acuracias_validacao = []

# Loop pelos valores de ccp_alpha
for ccp_alpha in ccp_alphas:
    # Árvore de decisão com a profundidade de 4 e o ccp_alpha atual
    dtc_2 = DecisionTreeClassifier(max_depth=4, ccp_alpha=ccp_alpha, random_state=42)
    
    # Treine o modelo com os dados de treinamento
    dtc_2.fit(X_treino[top_3_variaveis], y_treino)
    
    # Calcule a acurácia na base de validação
    acuracia_validacao = dtc_2.score(X_validacao[top_3_variaveis], y_validacao)
    
    # Adicione a acurácia à lista de acurácias na base de validação
    acuracias_validacao.append(acuracia_validacao)

# Encontre o melhor ccp_alpha com base na maior acurácia na base de validação
melhor_ccp_alpha = ccp_alphas[np.argmax(acuracias_validacao)]
melhor_acuracia_validacao = max(acuracias_validacao)

print(f"O melhor ccp_alpha é {melhor_ccp_alpha:.4f}")
print(f"A melhor acurácia na base de validação é {melhor_acuracia_validacao:.4f}")

# Crie a árvore de decisão final com a profundidade máxima e o melhor ccp_alpha
final_tree = DecisionTreeClassifier(ccp_alpha=melhor_ccp_alpha, random_state=42)
final_tree.fit(X_treino[top_3_variaveis], y_treino)

O melhor ccp_alpha é 0.0000
A melhor acurácia na base de validação é 0.8009


DecisionTreeClassifier(random_state=42)

### 5. Avaliação do modelo

Avalie a árvore encontrada no item anterior na base de testes.

Com a árvore final, foi obtido um ccp_alpha muito baixo. Podendo ser derivado de um overfitting da árvore, mesmo que este seja o ccp_alpha com maior acurácia. Seria melhor testar modelos com um ccp_alpha mais robusto, pensando em manter um modelo com uma complexidade e overfitting menor. Mesmo que a acurácia seja um pouco menor.